In [5]:
e = "fernet error traceback"
error_msg = "Invalid license. Please contact your system administrator for licensing Silverthread software. " \
            "Exception is as follows: \n\n" + e
print(error_msg)

Invalid license. Please contact your system administrator for licensing Silverthread software. Exception is as follows: 

fernet error traceback


In [11]:
from cryptography import fernet
import traceback

try:
    fernet.bleh()

except AttributeError as e:
    error_msg = "Invalid license. Please contact your system administrator for licensing Silverthread software. " \
                "Exception is as follows: \n\n {e}".format(e=e)
    print(error_msg)
    print(traceback.format_exc())

Invalid license. Please contact your system administrator for licensing Silverthread software. Exception is as follows: 

 module 'cryptography.fernet' has no attribute 'bleh'
Traceback (most recent call last):
  File "D:\Users\priscillaw\AppData\Local\Temp\1\ipykernel_12092\1348908338.py", line 5, in <module>
    fernet.bleh()
AttributeError: module 'cryptography.fernet' has no attribute 'bleh'



<hr>

In [2]:
import base64
from cryptography import fernet
from datetime import datetime, timezone
import re
import json
import os
import sys
import subprocess
from colorama import init as colorama_init
import logging
from colorama import init, Fore, Back, Style
init(autoreset=True)

def run(cmd):
    try:
        return subprocess.run(cmd, shell=True, capture_output=True, check=True, encoding="utf-8") \
                .stdout \
                .strip()
    except:
        return None


def _get_system_fingerprint() -> str:
    if sys.platform == 'win32' or sys.platform == 'cygwin' or sys.platform == 'msys':
        output = run('wmic csproduct get uuid')

        # UUIDs on Windows are always 36 digits long 
        # 32 hexadecimal digits (4 bits per digit) == 128-bit total + 4 hyphens
        return re.findall(r'[a-fA-F0-9\-]{36}', output)[0]

    if sys.platform.startswith('linux'):
        return run('cat /var/lib/dbus/machine-id') \
            or run('cat /etc/machine-id')


def _decrypt_hash(hash: str) -> bytes:
    key = b'T9DjWzbrlf4_Bdui4tmIB1XsA-PuW50BKj9wGmB1eLI='
    f = fernet.Fernet(key)
    try:
        return f.decrypt(str.encode(hash))
    except fernet.InvalidToken:
        raise Exception(
            "The contents of the modernize license file is invalid and cannot be interpreted. " +
            "Please contact your administrator."
        )


def _base64_decode(encoded_string):
    encoded_string_bytes = encoded_string.encode("ascii")

    decoded_string_bytes = base64.b64decode(encoded_string_bytes)
    return decoded_string_bytes.decode("ascii")


def _get_file_path():
    modernize_license_env_var = "CMRI_MOD_LIC"
    default_file_name = "modernize.lic"

    if modernize_license_env_var not in os.environ:
        raise Exception(
            f'''Modernize did not detect the environment variable ${modernize_license_env_var} to be set. ''' +
            "Please set this variable to the location of the folder that holds your modernize.lic file"
        )
    else:
        file_path = os.environ[modernize_license_env_var]

    file_path = os.path.normpath(file_path)

    if os.path.isdir(file_path):
        file_path = os.path.join(file_path, default_file_name)

    if not os.path.exists(file_path):
        raise Exception(
            f"The environment variable ${modernize_license_env_var} is set, but no license file was found at the"
            f" location specified. Please update the location or move the file to match"
        )

    return file_path


def _load_key_file():
    license_file_path = _get_file_path()

    with open(license_file_path, "r") as file:
        license_data = file.read()
    return json.loads(license_data)


def _get_system_time():
    return datetime.now(timezone.utc)


def _get_from_license(license: dict, key: str) -> str:
    if key not in license:
        raise Exception(f'''License file missing {key} key -- Please contact your administrator''')
    base64_encoded_value = license[key]

    encrypted_value = _base64_decode(base64_encoded_value)
    byte_value = _decrypt_hash(encrypted_value)
    return byte_value.decode("ascii")


def _check_license_timestamp(license):
    license_timestamp = _get_from_license(license, "timestamp")
    if _get_system_time() >= datetime.fromtimestamp(float(license_timestamp), tz=timezone.utc):
        raise Exception("CMRI key out of date -- Please contact your administrator")


def _check_license_fingerprint(license):
    license_fingerprint = _get_from_license(license, "fingerprint")
    if _get_system_fingerprint() != license_fingerprint:
        raise Exception("CMRI not licensed on this machine -- Please contact your administrator")


def check_license():
    license_key = _load_key_file()
    _check_license_timestamp(license_key)
    _check_license_fingerprint(license_key)


class ColorFormatter(logging.Formatter):
    # Change this dictionary to suit your coloring needs!
    COLORS = {
        "WARNING": Fore.RED,
        "ERROR": Fore.RED + Style.BRIGHT,
        "DEBUG": Fore.BLUE,
        "INFO": Fore.GREEN,
        "CRITICAL": Fore.RED
    }

    def format(self, record):
        color = self.COLORS.get(record.levelname, "")
        if color:
            record.name = color + record.name
            record.levelname = color + record.levelname
            record.msg = color + record.msg
        return logging.Formatter.format(self, record)

class ModernizeLoggerSimple:
    def __init__(self):
        colorama_init()
        self.console = logging.StreamHandler(sys.stderr)
        self.console.setFormatter(ColorFormatter("%(name)-10s %(levelname)-18s %(message)s"))
        self.console.setLevel(logging.DEBUG)

        self.logger = logging.getLogger("modernize-simple")
        self.logger.setLevel(logging.DEBUG)

        self.logger.addHandler(self.console)
        self.logger.propagate = False


    def log_error(self, msg):
        self.logger.error(f"{msg}")

    def log_info(self, msg):
        self.logger.info(f"{msg}")

    def log_warning(self, msg):
        self.logger.warning(f"{msg}")



#####################


mlogger_simple = ModernizeLoggerSimple()

try:
    mlogger_simple.log_info("Checking license...")
    check_license()
except Exception as exc:
    mlogger_simple.log_error(exc)
    #sys.exit()


modernize-simple INFO          Checking license...
modernize-simple INFO          Checking license...
modernize-simple ERROR    The contents of the modernize license file is invalid and cannot be interpreted. Please contact your administrator.
modernize-simple ERROR    The contents of the modernize license file is invalid and cannot be interpreted. Please contact your administrator.
